# Time To Event

In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'

## Time to event for W pages with external links

In [2]:
w_event_time_query = """
SELECT action, 
    CAST(AVG(event_offset_time) AS DECIMAL(10,2)) AS average_time_to_event_in_ms, 
    PERCENTILE(event_offset_time,0.5) AS median_time_to_event_in_ms,
    STDDEV(event_offset_time) as stddev,
    (PERCENTILE(event_offset_time,0.75) - PERCENTILE(event_offset_time,0.25)) as iqr

FROM 
    citationusage 
WHERE wiki = 'enwiki'
    AND page_id IN 
        (SELECT page_id FROM ryanmax.population_w_pages_with_extlinks)
    {}
    AND to_date(event_time) >= '{}'
    AND to_date(event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY action
"""

spark.sql(
    w_event_time_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    )).show()

+--------+---------------------------+--------------------------+--------------------+---------+
|  action|average_time_to_event_in_ms|median_time_to_event_in_ms|              stddev|      iqr|
+--------+---------------------------+--------------------------+--------------------+---------+
| fnHover|                 1791663.19|                   92090.0|2.8092415487170094E8| 289552.0|
| fnClick|                 1013807.62|                   70062.0| 1.250577069504599E7| 171408.0|
| upClick|                 1753665.93|                  181515.0| 1.712790104125731E7|545889.75|
|extClick|                  420234.13|                   35633.0|5.4244638925624825E7|  96515.0|
+--------+---------------------------+--------------------------+--------------------+---------+



## Time to event for WP:M pages with external links

In [3]:
wpm_event_time_query = """
SELECT action, 
    CAST(AVG(event_offset_time) AS DECIMAL(10,2)) AS average_time_to_event_in_ms, 
    PERCENTILE(event_offset_time,0.5) AS median_time_to_event_in_ms,
    STDDEV(event_offset_time) as stddev,
    (PERCENTILE(event_offset_time,0.75) - PERCENTILE(event_offset_time,0.25)) as iqr

FROM 
    citationusage 
WHERE wiki = 'enwiki'
    AND page_id IN 
        (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    {}
    AND to_date(event_time) >= '{}'
    AND to_date(event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY action
"""

spark.sql(
    wpm_event_time_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    )).show()

+--------+---------------------------+--------------------------+--------------------+--------+
|  action|average_time_to_event_in_ms|median_time_to_event_in_ms|              stddev|     iqr|
+--------+---------------------------+--------------------------+--------------------+--------+
| fnHover|                 6863961.98|                  123645.0| 2.487992256930276E9|458468.0|
| fnClick|                 1175430.43|                   91949.0|1.3628820880268283E7|242785.0|
| upClick|                 1866501.76|                  265348.0|1.7531366856536876E7|452209.0|
|extClick|                  889672.76|                   53675.0|1.2199096437395627E7|171432.0|
+--------+---------------------------+--------------------------+--------------------+--------+

